In [1]:
import MINING_D

**Add input output files and parameters to use.**

In [ ]:
input_cdr_file = '/Poppy/vbhardwaj/d_genes_project/Projects/PRJEB18631_sorted_mice_B_cells/' + \
            'Realigned/ERR1759678_divan/consensus_cdr3s.fasta'
output_file = './test_output/test_op.fasta'

**MINING-D parameters**

In [1]:
k=10
k_mers = 300
n_cores = 50
bidir_alpha = 0.5
cliq_th = 2
p_val_th = 4.5*10**(-36)

**Create a CDR3_rep object**

In [ ]:
cdr3s = CDR3_rep(input_cdr_file)

**Compute multiple extensions of 'k_mers' most abundant k-mers**

In [ ]:
extensions_dict = extensions_for_abundant_kmers(cdr3s, 
                                                num_k_mers_to_extend= k_mers, 
                                                n_cores=n_cores, 
                                                k_mer_len=k, 
                                                alpha= p_val_th)
filtered_extensions = filter_unidirectional_extensions(extensions_dict, side_alpha=bidir_alpha)

**Get initial candidate set**

In [ ]:
candidate_seqs = {ext for kmer in filtered_extensions for ext in filtered_extensions[kmer]}

# Create a D_gene_Candidates object 
cand = D_gene_Candidates()
cand.add_sequences(candidate_seqs)

**Add relative positions and keep only central candidates**

In [ ]:
cand.add_RP_in_cdrs_frm(cdr3s)
cand_central = cand.get_central_d_segments() 

**Filter candidates**

In [ ]:
# Filtering using merging cliques etc. 
cand_central = cand_central.del_abund_rand_insrtns()
cand_after_merging_cliques = cand_central.merge_cliques_in_db(a_cdr3_rep=cdr3s, k_mer_len=k, alpha= p_val_th,
                                                              graph_threshold=cliq_th)
cand_after_merging_cliques = cand_after_merging_cliques.\
                            del_abund_rand_insrtns(symbol_to_del = 'G',g_threshold=2).\
                            del_abund_rand_insrtns(symbol_to_del = 'C', g_threshold=2)

# Additional filtering for special cases (rare)
final_candidates = cand_after_merging_cliques.merge_smlr_seqs()
final_candidates = final_candidates.filter_cand_with_motif(motif = 'TACTACTAC')

**Write to Fasta**

In [5]:
final_candidates.write_to_fasta(output_file)